# DTOcean Installation Module Example

Note, this example assumes the Installation Module has been installed

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display, HTML

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [ ]:
import numpy as np

In [ ]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu, ThemeMenu
from dtocean_core.pipeline import Tree

In [ ]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [ ]:
# Bring up the logger
start_logging()

## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [ ]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
theme_menu = ThemeMenu()
pipe_tree = Tree()

## Create a new project

In [ ]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

## Set the device type

In [ ]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type"
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Wave Floating")
my_var.read(new_core, new_project)

## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [ ]:
project_menu.initiate_pipeline(new_core, new_project)

## Discover available modules

In [ ]:
names = module_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

## Activate a module

Note that the order of activation is important and that we can't deactivate yet!

In [ ]:
module_name = 'Installation'
module_menu.activate(new_core, new_project, module_name)

## Activate the Economics themes

In [ ]:
names = theme_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

In [ ]:
theme_menu.activate(new_core, new_project, "Economics")

## Check the status of the module inputs

In [ ]:
installation_branch = pipe_tree.get_branch(new_core, new_project, 'Installation')
input_status = installation_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [ ]:
project_menu.initiate_dataflow(new_core, new_project)

## Load test data

Prepare the test data for loading. The test_data directory of the source code should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [ ]:
%run test_data/inputs_wp5.py

In [ ]:
installation_branch.read_test_data(new_core,
                            new_project,
                            "test_data/inputs_wp5.pkl")

## Prepare the Economics Theme

In [ ]:
theme_name = "Economics"
eco_branch = pipe_tree.get_branch(new_core, new_project, "Economics")

input_status = eco_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Set the discount rate and cost estimates and project lifetime

In [ ]:
discount_rate = 0.05
device_cost = 1000000
project_lifetime = 20

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "project.discount_rate")
new_var.set_raw_interface(new_core, discount_rate)
new_var.read(new_core, new_project)

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "device.system_cost")
new_var.set_raw_interface(new_core, device_cost)
new_var.read(new_core, new_project)

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "project.lifetime")
new_var.set_raw_interface(new_core, project_lifetime)
new_var.read(new_core, new_project)

## Check if the module can be executed

In [ ]:
can_execute = module_menu.is_executable(new_core, new_project, module_name)
display(can_execute)

In [ ]:
input_status = installation_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [ ]:
module_menu.execute_current(new_core, new_project)

## Examine the results

In [ ]:
output_status = installation_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [ ]:
economics_data = new_core.get_data_value(new_project, "project.device_phase_installation_costs")
economics_data

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.device_phase_installation_cost_breakdown")
economics_data_breakdown

In [ ]:
port = new_core.get_data_value(new_project, "project.port")
port

In [ ]:
comp_cost = new_core.get_data_value(new_project, "project.electrical_phase_installation_costs")
comp_cost

In [ ]:
comp_time = new_core.get_data_value(new_project, "project.electrical_phase_installation_times")
comp_time

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.electrical_phase_installation_time_breakdown")
economics_data_breakdown

In [ ]:
comp_cost = new_core.get_data_value(new_project, "project.mooring_phase_installation_costs")
comp_cost

In [ ]:
comp_time = new_core.get_data_value(new_project, "project.mooring_phase_installation_times")
comp_time

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.mooring_phase_installation_time_breakdown")
economics_data_breakdown

In [ ]:
device_cost_breakdown = new_core.get_data_value(new_project, "project.device_phase_cost_class_breakdown")
electrical_cost_breakdown = new_core.get_data_value(new_project, "project.electrical_phase_installation_cost_breakdown")
mooring_cost_breakdown = new_core.get_data_value(new_project, "project.mooring_phase_installation_cost_breakdown")

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_phase_cost_breakdown")
economics_data_breakdown

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_cost_class_breakdown")
economics_data_breakdown

In [ ]:
device_time_breakdown = new_core.get_data_value(new_project, "project.device_phase_time_class_breakdown")
device_time_breakdown

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_phase_time_breakdown")
economics_data_breakdown

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_time_class_breakdown")
economics_data_breakdown

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_economics_data")
economics_data_breakdown

In [ ]:
output_status = eco_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.capex_breakdown")
economics_data_breakdown

In [ ]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.capex_total")
economics_data_breakdown